# Install additional Python libraries

In [1]:
# !pip install quantities

# Delete existing DB

In [2]:
!rm simoc_server/db.sqlite

# Import Python libraries

In [3]:
import json
from random import randint

In [4]:
import pprint
pp = pprint.PrettyPrinter()

In [5]:
from simoc_server import app, db
from simoc_server.database.db_model import User
from simoc_server.database.seed_data import seed
from simoc_server.game_runner import GameRunnerManager, GameRunnerInitializationParams

Unknown DB_TYPE variable: "None"
Using SQLite by default


In [6]:
from simoc_server.views import convert_configuration

In [7]:
import matplotlib.pyplot as plt
import numpy as np

# Load Agent model

In [8]:
db.drop_all()
db.create_all()
seed.seed('agent_desc.json')

# Create new user

In [9]:
username = 'user1'
password = 'user1'

user = User.query.filter_by(username=username).first()
if not user:
    print('Creating new user')
    user = User(username=username)
    user.set_password(password)
    db.session.add(user)
    db.session.commit()

Creating new user


# Initialize the ABM

In [10]:
game_config = {"agents": {
    "human_agent": [{"connections": {"air_storage": [1], "water_storage": [1, 2], "food_storage": [1]}, "amount": 10}],
    "cabbage": [{"connections": {"air_storage": [1], "water_storage": [1, 2], "nutrient_storage": [1],
                                 "power_storage": [1], "food_storage": [1]}, "amount": 10}],
    "lettuce": [{"connections": {"air_storage": [1], "water_storage": [1, 2], "nutrient_storage": [1],
                                 "power_storage": [1], "food_storage": [1]}, "amount": 10}],
    "greenhouse_medium": [{"connections": {"power_storage": [1]}, "amount": 1}],
    "solar_pv_array": [{"connections": {"power_storage": [1]}, "amount": 100}],
    "multifiltration_purifier_post_treament": [{"connections": {"water_storage": [1, 2]}, "amount": 1}],
    "urine_recycling_processor_VCD": [{"connections": {"power_storage": [1], "water_storage": [1, 2]}, "amount": 1}],
    "co2_removal_SAWD": [{"connections": {"power_storage": [1], "air_storage": [1]}, "amount": 1}]},
"storages": {
    "air_storage": [{"id": 1, "atmo_h2o": 100, "atmo_o2": 100, "atmo_co2": 100}],
    "water_storage": [{"id": 1, "h2o_potb": 100, "h2o_tret": 100}, {"id": 2, "h2o_wste": 100, "h2o_urin": 0}],
    "nutrient_storage": [{"id": 1, "sold_n": 100, "sold_p": 100, "sold_k": 100}],
    "power_storage": [{"id": 1, "enrg_kwh": 1000}],
    "food_storage": [{"id": 1, "food_edbl": 1000}]},
"termination": {
    "time": {"value": 2000, "unit": "hour"}},
    "food_leaf": {"value": 10000, "unit": "kg"},
    "evacuation": {}
}

In [11]:
start_data = {"game_config": {'logging': {}, 'duration': {'value': 30, 'type': 'day'}, 'human_agent': {'amount': 4}, 'habitat': 'crew_habitat_small', 'greenhouse': 'greenhouse_small', 'food_storage': {'amount': 1000}, 'solar_arrays': {'amount': 60}, 'power_storage': {'amount': 1}, 'plants': [{'species': 'rice', 'amount': 100}, {'species': 'wheat', 'amount': 30}, {'species': 'cabbage', 'amount': 4}, {'species': 'rice', 'amount': 100}, {'species': 'wheat', 'amount': 30}, {'species': 'cabbage', 'amount': 3}]}}
# start_data = {"game_config": {"priorities": ["inhabitants", "eclss", "plants", "storage"], 'logging': {'columns': ['agent_id', 'agent_type', 'value', 'unit'], 'filters': [('currency', ['enrg_kwh']), ('direction', ['in'])]}, 'duration': {'value': 30, 'type': 'day'}, 'human_agent': {'amount': 4}, 'habitat': 'crew_habitat_small', 'greenhouse': 'greenhouse_small', 'food_storage': {'amount': 1000}, 'solar_arrays': {'amount': 60}, 'power_storage': {'amount': 1}, 'plants': [{'species': 'rice', 'amount': 100}, {'species': 'wheat', 'amount': 30}, {'species': 'cabbage', 'amount': 4}, {'species': 'rice', 'amount': 100}, {'species': 'wheat', 'amount': 30}, {'species': 'cabbage', 'amount': 3}]}}
# start_data = {"game_config": {'logging': {'columns': ['agent_id', 'agent_type', 'value', 'unit'], 'filters': [('currency', ['enrg_kwh']), ('direction', ['in'])]}, 'duration': {'value': 30, 'type': 'day'}, 'human_agent': {'amount': 4}, 'habitat': 'crew_habitat_small', 'greenhouse': 'greenhouse_small', 'food_storage': {'amount': 1000}, 'solar_arrays': {'amount': 60}, 'power_storage': {'amount': 1}, 'plants': [{'species': 'rice', 'amount': 100}, {'species': 'wheat', 'amount': 30}, {'species': 'cabbage', 'amount': 4}, {'species': 'rice', 'amount': 100}, {'species': 'wheat', 'amount': 30}, {'species': 'cabbage', 'amount': 3}]}}
# start_data = {"game_config": {'duration': {'value': 30, 'type': 'day'}, 'human_agent': {'amount': 4}, 'habitat': 'crew_habitat_small', 'greenhouse': 'greenhouse_small', 'food_storage': {'amount': 1000}, 'solar_arrays': {'amount': 60}, 'power_storage': {'amount': 1}, 'plants': [{'species': 'rice', 'amount': 100}, {'species': 'wheat', 'amount': 30}, {'species': 'cabbage', 'amount': 4}, {'species': 'rice', 'amount': 100}, {'species': 'wheat', 'amount': 30}, {'species': 'cabbage', 'amount': 3}]}}
pp.pprint(start_data)

{'game_config': {'duration': {'type': 'day', 'value': 30},
                 'food_storage': {'amount': 1000},
                 'greenhouse': 'greenhouse_small',
                 'habitat': 'crew_habitat_small',
                 'human_agent': {'amount': 4},
                 'logging': {},
                 'plants': [{'amount': 100, 'species': 'rice'},
                            {'amount': 30, 'species': 'wheat'},
                            {'amount': 4, 'species': 'cabbage'},
                            {'amount': 100, 'species': 'rice'},
                            {'amount': 30, 'species': 'wheat'},
                            {'amount': 3, 'species': 'cabbage'}],
                 'power_storage': {'amount': 1},
                 'solar_arrays': {'amount': 60}}}


In [12]:
game_config = convert_configuration(start_data["game_config"])
pp.pprint(game_config)

{'agents': {'cabbage': [{'amount': 3,
                         'connections': {'air_storage': [1],
                                         'food_storage': [1],
                                         'nutrient_storage': [1],
                                         'power_storage': [1],
                                         'water_storage': [1, 2]}}],
            'co2_reduction_sabatier': [{'amount': 1,
                                        'connections': {'air_storage': [1],
                                                        'power_storage': [1],
                                                        'water_storage': [1,
                                                                          2]}}],
            'co2_removal_SAWD': [{'amount': 1,
                                  'connections': {'air_storage': [1],
                                                  'power_storage': [1]}}],
            'crew_habitat_small': [{'amount': 1,
                                   

In [13]:
game_runner_manager = GameRunnerManager()
game_runner_init_params = GameRunnerInitializationParams(game_config)
game_runner_manager.new_game(user, game_runner_init_params)
game_runner = game_runner_manager.get_game_runner(user)

# Run the simiulation

In [14]:
def run_sim(game_runner_manager, user, num_steps):
    for i in range(1, num_steps + 1):
        agent_model_state = game_runner_manager.get_step(user, i)
    pp.pprint(agent_model_state)

In [15]:
%%time
run_sim(game_runner_manager, user, 10)

{'agents': {'total_agent_types': {'cabbage': 3,
                                  'co2_reduction_sabatier': 1,
                                  'co2_removal_SAWD': 1,
                                  'crew_habitat_small': 1,
                                  'greenhouse_small': 1,
                                  'human_agent': 4,
                                  'multifiltration_purifier_post_treament': 1,
                                  'oxygen_generation_SFWE': 1,
                                  'rice': 100,
                                  'solar_pv_array_mars': 60,
                                  'solid_waste_aerobic_bioreactor': 1,
                                  'urine_recycling_processor_VCD': 1,
                                  'wheat': 30},
            'total_consumption': {'atmo_co2': {'units': 'kg',
                                               'value': '0.0848'},
                                  'atmo_h2': {'units': 'kg', 'value': '0.0016'},
               

                0.0011201294643],
               ['GeneralAgent_7fea6615',
                'rice',
                'food_edbl',
                'out',
                1,
                'food_storage',
                '1.0 kg',
                0.0],
               ['GeneralAgent_5989cb0a',
                'wheat',
                'atmo_co2',
                'in',
                1,
                'air_storage',
                '1.0 kg',
                0.002547123016],
               ['GeneralAgent_5989cb0a',
                'wheat',
                'h2o_potb',
                'in',
                1,
                'water_storage',
                '1.0 kg',
                0.005625],
               ['GeneralAgent_5989cb0a',
                'wheat',
                'h2o_potb',
                'in',
                2,
                'water_storage',
                '1.0 kg',
                0.005625],
               ['GeneralAgent_5989cb0a',
                'wheat',
                '

                'atmo_co2',
                'in',
                1,
                'air_storage',
                '1.0 kg',
                7.552675700833742e-06],
               ['GeneralAgent_b0969e61',
                'rice',
                'h2o_potb',
                'in',
                1,
                'water_storage',
                '1.0 kg',
                0.0049943287027463985],
               ['GeneralAgent_b0969e61',
                'rice',
                'h2o_potb',
                'in',
                2,
                'water_storage',
                '1.0 kg',
                0.0049943287027463985],
               ['GeneralAgent_b0969e61',
                'rice',
                'sold_n',
                'in',
                1,
                'nutrient_storage',
                '1.0 kg',
                2.06755062e-05],
               ['GeneralAgent_b0969e61',
                'rice',
                'sold_p',
                'in',
                1,
         

                0.0049943287027463985],
               ['GeneralAgent_46de85a6',
                'rice',
                'sold_n',
                'in',
                1,
                'nutrient_storage',
                '1.0 kg',
                2.06755062e-05],
               ['GeneralAgent_46de85a6',
                'rice',
                'sold_p',
                'in',
                1,
                'nutrient_storage',
                '1.0 kg',
                3.270886e-07],
               ['GeneralAgent_46de85a6',
                'rice',
                'sold_k',
                'in',
                1,
                'nutrient_storage',
                '1.0 kg',
                2.466768e-06],
               ['GeneralAgent_46de85a6',
                'rice',
                'enrg_kwh',
                'in',
                1,
                'power_storage',
                '1.0 kWh',
                0.11504350735],
               ['GeneralAgent_46de85a6',
                

                'enrg_kwh',
                'in',
                1,
                'power_storage',
                '1.0 kWh',
                0.11504350735],
               ['GeneralAgent_929c456f',
                'rice',
                'atmo_o2',
                'out',
                1,
                'air_storage',
                '1.0 kg',
                0.0011201294643],
               ['GeneralAgent_929c456f',
                'rice',
                'food_edbl',
                'out',
                1,
                'food_storage',
                '1.0 kg',
                0.0],
               ['GeneralAgent_ba471148',
                'rice',
                'atmo_co2',
                'in',
                1,
                'air_storage',
                '1.0 kg',
                7.552675700833742e-06],
               ['GeneralAgent_ba471148',
                'rice',
                'h2o_potb',
                'in',
                1,
                'water_storage',


                'rice',
                'atmo_co2',
                'in',
                1,
                'air_storage',
                '1.0 kg',
                7.552675700833742e-06],
               ['GeneralAgent_e58c5a4e',
                'rice',
                'h2o_potb',
                'in',
                1,
                'water_storage',
                '1.0 kg',
                0.0049943287027463985],
               ['GeneralAgent_e58c5a4e',
                'rice',
                'h2o_potb',
                'in',
                2,
                'water_storage',
                '1.0 kg',
                0.0049943287027463985],
               ['GeneralAgent_e58c5a4e',
                'rice',
                'sold_n',
                'in',
                1,
                'nutrient_storage',
                '1.0 kg',
                2.06755062e-05],
               ['GeneralAgent_e58c5a4e',
                'rice',
                'sold_p',
                'in',
    

                'air_storage',
                '1.0 kg',
                7.552675700833742e-06],
               ['GeneralAgent_e7429ae6',
                'rice',
                'h2o_potb',
                'in',
                1,
                'water_storage',
                '1.0 kg',
                0.0049943287027463985],
               ['GeneralAgent_e7429ae6',
                'rice',
                'h2o_potb',
                'in',
                2,
                'water_storage',
                '1.0 kg',
                0.0049943287027463985],
               ['GeneralAgent_e7429ae6',
                'rice',
                'sold_n',
                'in',
                1,
                'nutrient_storage',
                '1.0 kg',
                2.06755062e-05],
               ['GeneralAgent_e7429ae6',
                'rice',
                'sold_p',
                'in',
                1,
                'nutrient_storage',
                '1.0 kg',
                

In [16]:
from simoc_server.agent_model.agents.core import GeneralAgent, StorageAgent

storages = game_runner.agent_model.get_agents_by_class(agent_class=StorageAgent)

In [17]:
from simoc_server.database.db_model import AgentType

agent_class = AgentType.query.get(storages[0]._agent_type_id).agent_class

In [18]:
import pandas as pd

In [19]:
def get_step_logs(step_num, filters=[], logs=None, columns=None, dtype='list'):
    df = pd.DataFrame(logs)
    df = df.loc[df['step_num'] == step_num].drop('step_num', axis=1)
    for col, val in filters:
        df = df.loc[df[col].isin(val)].drop(col, axis=1)
    if columns:
        df = df.loc[:, columns]
    if dtype == 'list':
        return df.values.tolist()
    elif dtype == 'dict':
        return df.to_dict(orient='records')
    elif dtype == 'df':
        return df

logs = game_runner.agent_model.logs
print(len(logs))

25327


In [46]:
step_num = None

# filters = [('currency', ['enrg_kwh']), ('direction', ['in'])]
filters = []

# cols = ['agent_id', 'unit']
cols = []

df = game_runner.agent_model.get_step_logs(step_num, filters, columns=cols, dtype='df')

In [47]:
df.head()

,agent_id,agent_type,currency,direction,storage_id,storage_type,unit,value


In [21]:
df.columns

Index(['agent_id', 'agent_type', 'currency', 'direction', 'storage_id',
       'storage_type', 'unit', 'value'],
      dtype='object')

In [35]:
df1 = df.loc[:,['agent_type', 'agent_id', 'currency', 'direction', 'storage_id', 'storage_type', 'value', 'unit']]

In [40]:
df.values.tolist()

[['GeneralAgent_54c9f3ce',
  'rice',
  'atmo_co2',
  'in',
  1,
  'air_storage',
  '1.0 kg',
  7.552675700833742e-07],
 ['GeneralAgent_54c9f3ce',
  'rice',
  'h2o_potb',
  'in',
  1,
  'water_storage',
  '1.0 kg',
  0.004999432580594009],
 ['GeneralAgent_54c9f3ce',
  'rice',
  'h2o_potb',
  'in',
  2,
  'water_storage',
  '1.0 kg',
  0.004999432580594009],
 ['GeneralAgent_54c9f3ce',
  'rice',
  'sold_n',
  'in',
  1,
  'nutrient_storage',
  '1.0 kg',
  2.06755062e-05],
 ['GeneralAgent_54c9f3ce',
  'rice',
  'sold_p',
  'in',
  1,
  'nutrient_storage',
  '1.0 kg',
  3.270886e-07],
 ['GeneralAgent_54c9f3ce',
  'rice',
  'sold_k',
  'in',
  1,
  'nutrient_storage',
  '1.0 kg',
  2.466768e-06],
 ['GeneralAgent_54c9f3ce',
  'rice',
  'enrg_kwh',
  'in',
  1,
  'power_storage',
  '1.0 kWh',
  0.11504350735],
 ['GeneralAgent_54c9f3ce',
  'rice',
  'atmo_o2',
  'out',
  1,
  'air_storage',
  '1.0 kg',
  0.0011201294643],
 ['GeneralAgent_54c9f3ce',
  'rice',
  'food_edbl',
  'out',
  1,
  'food

In [22]:
df.agent_type.value_counts()

rice                                      900
wheat                                     270
solar_pv_array_mars                        60
human_agent                                40
cabbage                                    27
solid_waste_aerobic_bioreactor             11
co2_reduction_sabatier                      6
multifiltration_purifier_post_treament      5
urine_recycling_processor_VCD               5
oxygen_generation_SFWE                      5
co2_removal_SAWD                            2
greenhouse_small                            1
crew_habitat_small                          1
Name: agent_type, dtype: int64

In [23]:
df.loc[df['agent_type'] == 'rice'].head()

,agent_id,agent_type,currency,direction,storage_id,storage_type,unit,value
0,GeneralAgent_54c9f3ce,rice,atmo_co2,in,1,air_storage,1.0 kg,7.552676e-07
1,GeneralAgent_54c9f3ce,rice,h2o_potb,in,1,water_storage,1.0 kg,4.999433e-03
2,GeneralAgent_54c9f3ce,rice,h2o_potb,in,2,water_storage,1.0 kg,4.999433e-03
3,GeneralAgent_54c9f3ce,rice,sold_n,in,1,nutrient_storage,1.0 kg,2.067551e-05
4,GeneralAgent_54c9f3ce,rice,sold_p,in,1,nutrient_storage,1.0 kg,3.270886e-07


In [24]:
# game_runner.agent_model.get_logs(filters=filters, columns=cols, dtype='list')[:5]
game_runner.agent_model.get_logs(filters=filters, columns=cols, dtype='df').head()

,agent_id,agent_type,currency,direction,step_num,storage_id,storage_type,unit,value
0,GeneralAgent_54c9f3ce,rice,atmo_co2,in,0,1,air_storage,1.0 kg,7.552676e-07
1,GeneralAgent_54c9f3ce,rice,h2o_potb,in,0,1,water_storage,1.0 kg,4.999433e-03
2,GeneralAgent_54c9f3ce,rice,h2o_potb,in,0,2,water_storage,1.0 kg,4.999433e-03
3,GeneralAgent_54c9f3ce,rice,sold_n,in,0,1,nutrient_storage,1.0 kg,2.067551e-05
4,GeneralAgent_54c9f3ce,rice,sold_p,in,0,1,nutrient_storage,1.0 kg,3.270886e-07


In [25]:
# import cProfile
# cProfile.run('run_sim(game_runner_manager, user, 10)', filename='simoc.cprof' )

In [26]:
# from profiling.tracing import TracingProfiler
# profiler = TracingProfiler()
# profiler.start()
# run_sim(game_runner_manager, user, 10)
# profiler.stop()

In [27]:
# profiler.run_viewer()

In [28]:
# profiler.dump('simoc.profiler')

In [29]:
# game_runner_manager.save_all(allow_repeat_save=False)

In [30]:
# game_runner.agent_model.get_total_storages()

In [31]:
# game_runner.agent_model.get_total_agents()

In [32]:
# game_runner.agent_model.get_model_stats()